In [7]:
import numpy as np                 #for general calculations.
import matplotlib.pyplot as plt    #making plots.
import xarray as xr                #loading and working with data in an optimal way.

from matplotlib.animation import FuncAnimation  #creating animations and videos.
from IPython.display import Video, HTML         #embedding videos in the notebook

import cartopy.crs as ccrs         # to plot maps with different projections
import cartopy.feature as cfeature # to plot coastlines, land, borders, etc.

In [8]:
ds = xr.load_dataset("./data/nc/mixed_layer_properties_mean.nc")
latitude_bounds = slice(45, 50)
longitude_bounds = slice(325, 335)

In [9]:
ds_filtered = ds.sel(latitude = latitude_bounds, longitude = longitude_bounds)
weights = np.cos(np.deg2rad(ds.latitude)); weights.name = "weights"
ds_mean_unweighted = ds_filtered.mean(dim=['latitude','longitude'])
ds_mean = ds_filtered.weighted(weights).mean(dim=['latitude','longitude'])
ds_mean["month_corr"] = [m - 0.5 for m in ds_mean["month"]]

In [10]:
# ds_mean.to_netcdf("./data/nc/mixed_layer_processed.nc")

In [11]:
depth_video=-ds_filtered['depth_mean']

def animation_func(i):
    fig.clear()
    ax = plt.axes(projection=ccrs.PlateCarree())                    #specifying the projection such that coasline data could be added
    depth_video.isel(month=i).plot(cmap='RdBu_r',vmin=-200, vmax=0) #plotting the data
    ax.add_feature(cfeature.COASTLINE)                              #adding the coastline over the plotted data
    ax.add_feature(cfeature.BORDERS)                                #adding the borders
    plt.axis('off')

fig = plt.figure(figsize=(10,4))
animation = FuncAnimation(fig, animation_func, 
                          frames = len(depth_video.month))

animation.save('depth.mp4') 
plt.close()